# CPC 2023 SWB Ephys Analysis

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import emd
import re
import scipy

import warnings
warnings.filterwarnings('ignore')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
import sys
sys.path.append('/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/ephys_analysis/scripts/saez_python_pipeline/LFPAnalysis')

In [4]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

## Load epoched data and subj info

In [6]:
swb_dir = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/'
anat_dir = f'{swb_dir}anat_recons/recon_labels/'
neural_dir = f'{swb_dir}ephys_analysis/data/'
behav_dir = f'{swb_dir}behavior_analysis/behavior_preprocessed/'

In [7]:
subj_list = pd.read_excel(f'{swb_dir}presentations/cpc_2023_poster/subjects.xlsx', sheet_name='patient_info', usecols=[0])
subj_ids = list(subj_list.PatientID)

In [17]:
#dictionaries for data storage
elec_dict = {f'{x}': [] for x in subj_ids}
behav_dict = {f'{x}': [] for x in subj_ids}
evs = {'DecisionOnset': [0, 3.0],
       'TrialOnset': [-1.0, 0]}
epochs_all_subjs_all_evs = {f'{a}': {f'{b}': np.nan for b in evs.keys()} for a in subj_ids}


In [18]:
#load epoched data
for subj_id in subj_ids:
    ## Load the electrode localization data & save to group dict
    anat_file = glob(f'{anat_dir}{subj_id}_labels.csv')[0]
    elec_locs = pd.read_csv(anat_file)
    elec_locs = elec_locs[elec_locs.columns.drop(list(elec_locs.filter(regex='Unnamed')))]
    elec_dict[subj_id] = elec_locs
    ## Load behavioral data & save to group dict
    behav_df = pd.read_csv(f'{behav_dir}{subj_id}_pt_task_data')
    behav_df = behav_df[behav_df.columns.drop(list(behav_df.filter(regex='Unnamed')))]
    behav_dict[subj_id] = behav_df

    for event in evs.keys():
        
        ev_epochs = mne.read_epochs(f'{neural_dir}{subj_id}/epoch_{event}.fif', preload=True)
        
        epochs_all_subjs_all_evs[subj_id][event] = ev_epochs

## Wavelet TFRs

In [19]:
analysis_evs = ['DecisionOnset']
baseline_ev = 'TrialOnset'
evs = {'DecisionOnset': [0, 3.0],
       'TrialOnset': [-1.0, 0]}

for i in analysis_evs: 
    if i not in evs.keys(): 
        raise ValueError('This analysis event is not in the dictionary')

In [20]:
# Set some spectrogram parameters 
freqs = np.logspace(*np.log10([4, 200]), num=30)
n_cycles = 4  
sr = 500
buf = 1.0
buf_ix = int(buf*sr)


baseline_power_epochs = {f'{a}': {f'{b}': np.nan for b in evs.keys()} for a in subj_ids}

power_epochs = {f'{a}': {f'{b}': np.nan for b in evs.keys()} for a in subj_ids}

In [21]:
# Compute baseline_power
for subj_id in subj_ids:
    event = 'TrialOnset'
    epochs = epochs_all_subjs_all_evs[subj_id][event]
        
    # Let's make sure we only do this for good channels
    good_chans = [x for x in epochs.ch_names if x not in epochs.info['bads']]
    picks = [x for x in good_chans]

    pow_struct = np.nan * np.ones([epochs._data.shape[0], 
                           epochs._data.shape[1], len(freqs), 
                           epochs._data.shape[-1]])

    for ch_ix in np.arange(epochs._data.shape[1]): 
        ch_data = epochs._data[:, ch_ix:ch_ix+1, :]
        bad_epochs  = np.where(epochs.metadata[epochs.ch_names[ch_ix]].notnull())[0]
        good_epochs = np.delete(np.arange(ch_data.shape[0]), bad_epochs)
        ch_data = np.delete(ch_data, bad_epochs, axis=0)
        ch_pow = mne.time_frequency.tfr_array_morlet(ch_data, sfreq=epochs.info['sfreq'], 
                                            freqs=freqs, n_cycles=n_cycles, zero_mean=False, 
                                            use_fft=True, output='power', n_jobs=1)

        pow_struct[good_epochs, ch_ix, :, :] = ch_pow[:, 0, :, :]

    temp_pow = mne.time_frequency.EpochsTFR(epochs.info, pow_struct, 
                                            epochs.times, freqs)
    temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

    baseline_power_epochs[subj_id][event] = temp_pow

In [22]:
baseline_power_epochs

{'DA8': {'DecisionOnset': nan,
  'TrialOnset': <EpochsTFR | time : [-1.000000, 0.000000], freq : [4.000000, 200.000000], epochs : 150, channels : 57, ~1004.0 MB>},
 'DA9': {'DecisionOnset': nan,
  'TrialOnset': <EpochsTFR | time : [-1.000000, 0.000000], freq : [4.000000, 200.000000], epochs : 150, channels : 44, ~756.9 MB>},
 'DA10': {'DecisionOnset': nan,
  'TrialOnset': <EpochsTFR | time : [-1.000000, 0.000000], freq : [4.000000, 200.000000], epochs : 150, channels : 41, ~705.3 MB>},
 'DA11': {'DecisionOnset': nan,
  'TrialOnset': <EpochsTFR | time : [-1.000000, 0.000000], freq : [4.000000, 200.000000], epochs : 151, channels : 91, ~1.54 GB>},
 'DA023': {'DecisionOnset': nan,
  'TrialOnset': <EpochsTFR | time : [-1.000000, 0.000000], freq : [4.000000, 200.000000], epochs : 150, channels : 125, ~2.15 GB>},
 'MS002': {'DecisionOnset': nan,
  'TrialOnset': <EpochsTFR | time : [-1.000000, 0.000000], freq : [4.000000, 200.000000], epochs : 150, channels : 125, ~2.15 GB>},
 'MS003': {'Deci

In [23]:
# load pickle module
import pickle

save_path = f'{swb_dir}presentations/cpc_2023_poster/data/'

# create a binary pickle file 
f = open(f'{save_path}/epochs_07032023.pkl',"wb")

# write the python object (dict) to pickle file
pickle.dump(epochs_all_subjs_all_evs,f)

# close file
f.close()

# create a binary pickle file 
f = open(f'{save_path}/baseline_epochs_07032023.pkl',"wb")

# write the python object (dict) to pickle file
pickle.dump(baseline_power_epochs,f)

# close file
f.close()

In [10]:
import pickle
save_path = f'{swb_dir}presentations/cpc_2023_poster/data/'

epochs_all_subjs_all_evs = pd.read_pickle(rf'{save_path}epochs_07032023.pkl')
baseline_power_epochs = pd.read_pickle(rf'{save_path}baseline_epochs_07032023.pkl')

In [24]:
subj_ids

['DA8',
 'DA9',
 'DA10',
 'DA11',
 'DA023',
 'MS002',
 'MS003',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS025',
 'MS026',
 'MS027',
 'MS029',
 'MS030']

In [25]:
power_epochs = {f'{a}': {f'{b}': np.nan for b in evs.keys()} for a in subj_ids}

In [28]:
subj_id = 'MS030'
print(subj_id)
event = 'DecisionOnset'        
epochs = epochs_all_subjs_all_evs[subj_id][event]
            
# Let's make sure we only do this for good channels
good_chans = [x for x in epochs.ch_names if x not in epochs.info['bads']]
picks = [x for x in good_chans]

pow_struct = np.nan * np.ones([epochs._data.shape[0], 
                        epochs._data.shape[1], len(freqs), 
                        epochs._data.shape[-1]])

for ch_ix in np.arange(epochs._data.shape[1]): 
    ch_data = epochs._data[:, ch_ix:ch_ix+1, :]
    bad_epochs  = np.where(epochs.metadata[epochs.ch_names[ch_ix]].notnull())[0]
    good_epochs = np.delete(np.arange(ch_data.shape[0]), bad_epochs)
    ch_data = np.delete(ch_data, bad_epochs, axis=0)
    ch_pow = mne.time_frequency.tfr_array_morlet(ch_data, sfreq=epochs.info['sfreq'], 
                                        freqs=freqs, n_cycles=n_cycles, zero_mean=False, 
                                        use_fft=True, output='power', n_jobs=1)

    pow_struct[good_epochs, ch_ix, :, :] = ch_pow[:, 0, :, :]

temp_pow = mne.time_frequency.EpochsTFR(epochs.info, pow_struct, 
                                        epochs.times, freqs)

temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

baseline_corrected_power = lfp_preprocess_utils.zscore_TFR_across_trials(temp_pow.data, 
                                baseline_power_epochs[subj_id]['TrialOnset'], mode='zscore', baseline_only=False)

zpow = mne.time_frequency.EpochsTFR(epochs.info, baseline_corrected_power, 
                                temp_pow.times, freqs)

zpow.metadata = epochs_all_subjs_all_evs[subj_id][event].metadata

power_epochs[subj_id][event] = zpow

MS030


In [29]:
# load pickle module
import pickle

save_path = f'{swb_dir}presentations/cpc_2023_poster/data/'

# create a binary pickle file 
# f = open(f'{save_path}power_epochs_davis.pkl',"wb")
# f = open(f'{save_path}power_epochs_ms2toms16.pkl',"wb")
# f = open(f'{save_path}power_epochs_ms17toms22.pkl',"wb")
# f = open(f'{save_path}power_epochs_ms25toms27.pkl',"wb")
# f = open(f'{save_path}power_epochs_ms29toms30.pkl',"wb")
# f = open(f'{save_path}power_epochs_ms29.pkl',"wb") #had to redo - behavioral metadata didn't save originally
f = open(f'{save_path}power_epochs_ms30.pkl',"wb") #reran separately to save data

# write the python object (dict) to pickle file
pickle.dump(power_epochs,f)

# close file
f.close()


In [14]:
del power_epochs

In [15]:
del epochs_all_subjs_all_evs
del baseline_power_epochs

In [14]:
# #baseline correct analysis ev 
# for subj_id in subj_ids:
#     event = 'DecisionOnset'
            
#     epochs = epochs_all_subjs_all_evs[subj_id][event]
        
#     # Let's make sure we only do this for good channels
#     good_chans = [x for x in epochs.ch_names if x not in epochs.info['bads']]
#     picks = [x for x in good_chans]
    
#     pow_struct = np.nan * np.ones([epochs._data.shape[0], 
#                             epochs._data.shape[1], len(freqs), 
#                             epochs._data.shape[-1]])

#     for ch_ix in np.arange(epochs._data.shape[1]): 
#         ch_data = epochs._data[:, ch_ix:ch_ix+1, :]
#         bad_epochs  = np.where(epochs.metadata[epochs.ch_names[ch_ix]].notnull())[0]
#         good_epochs = np.delete(np.arange(ch_data.shape[0]), bad_epochs)
#         ch_data = np.delete(ch_data, bad_epochs, axis=0)
#         ch_pow = mne.time_frequency.tfr_array_morlet(ch_data, sfreq=epochs.info['sfreq'], 
#                                             freqs=freqs, n_cycles=n_cycles, zero_mean=False, 
#                                             use_fft=True, output='power', n_jobs=1)

#         pow_struct[good_epochs, ch_ix, :, :] = ch_pow[:, 0, :, :]
    
#     temp_pow = mne.time_frequency.EpochsTFR(epochs.info, pow_struct, 
#                                             epochs.times, freqs)
    
#     temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])
    
#     baseline_corrected_power = lfp_preprocess_utils.zscore_TFR_across_trials(temp_pow.data, 
#                                     baseline_power_epochs[subj_id]['TrialOnset'], mode='zscore', baseline_only=False)
    
#     zpow = mne.time_frequency.EpochsTFR(epochs.info, baseline_corrected_power, 
#                                     temp_pow.times, freqs)
    
#     zpow.metadata = epochs_all_subjs_all_evs[subj_id][event].metadata
    
#     power_epochs[subj_id][event] = zpow

: 

: 

# TFR Visualization

In [ ]:
load_path = f'{swb_dir}presentations/cpc_2023_poster/data/'
power_epochs_davis = pd.read_pickle(f'{load_path}power_epochs_davis.pkl')
power_epochs_ms2toms16 = pd.read_pickle(f'{load_path}power_epochs_ms2toms16.pkl')
power_epochs_ms17toms22 = pd.read_pickle(f'{load_path}power_epochs_ms17toms22.pkl')
power_epochs_ms25toms27 = pd.read_pickle(f'{load_path}power_epochs_ms25toms27.pkl')
power_epochs_ms29toms30 = pd.read_pickle(f'{load_path}power_epochs_ms29toms30.pkl')
power_epochs_ms29toms30 = pd.read_pickle(f'{load_path}power_epochs_ms29.pkl')


In [ ]:
power_epochs = {f'{a}': {f'{b}': np.nan for b in evs.keys()} for a in subj_ids}

In [ ]:
#load epoched data
for subj_id in subj_ids:
    ## Load the electrode localization data & save to group dict
    anat_file = glob(f'{anat_dir}{subj_id}_labels.csv')[0]
    elec_locs = pd.read_csv(anat_file)
    elec_locs = elec_locs[elec_locs.columns.drop(list(elec_locs.filter(regex='Unnamed')))]
    elec_dict[subj_id] = elec_locs
    ## Load behavioral data & save to group dict
    behav_df = pd.read_csv(f'{behav_dir}{subj_id}_pt_task_data')
    behav_df = behav_df[behav_df.columns.drop(list(behav_df.filter(regex='Unnamed')))]
    behav_dict[subj_id] = behav_df

    for event in evs.keys():
        
        ev_epochs = mne.read_epochs(f'{neural_dir}{subj_id}/epoch_{event}.fif', preload=True)
        
        epochs_all_subjs_all_evs[subj_id][event] = ev_epochs

In [ ]:
rois = ['vmPFC','Superior frontal', 'Middle frontal','Inferior frontal','Orbital frontal','Cingulate','Insula','Amygdala','Hippocampus']

In [ ]:
#dictionaries for data storage
elec_dict = {f'{x}': [] for x in subj_ids}
behav_dict = {f'{x}': [] for x in subj_ids}
evs = {'DecisionOnset': [0, 3.0],
       'TrialOnset': [-1.0, 0]}

In [ ]:
#tfrs
event = 'DecisionOnset'

# rois = ['hippocampus', 'amygdala', 'insular', 'cingulate' ,'orbital']
region = 'orbital'

# band definitions for y-axis
yticks = [4, 8, 13, 30, 60, 120]

# task condition to contrast 
#conditions = ["(outcome=='good')",
 #            "(outcome=='bad')"]

#cond_name = 'outcome'

# task condition to contrast 
conditions = ["(regret<0)",
              "(regret==0)"]


cond_name = 'regret'

for subj_id in subj_ids:
    # Get electrode df 
    elec_file = anat_path + subj_id + '_labels.csv'

    elec_data = lfp_preprocess_utils.load_elec(elec_file)

    anode_list = [x.split('-')[0] for x in epochs_all_subjs_all_evs[subj_id][event].ch_names]
    elec_df = elec_data[elec_data.label.str.lower().isin(anode_list)]
    elec_df['label'] = epochs_all_subjs_all_evs[subj_id][event].ch_names
    elec_data = elec_data.dropna(how='all')

    picks = analysis_utils.select_picks_rois(elec_df, region)
    
    for event in analysis_evs:
        fig, ax = plt.subplots(1, 2, figsize=(20, 6), dpi=300)
        for ix, cond in enumerate(conditions):
            # Set the times for 
            times = power_epochs[subj_id][event].times
            plot_data = np.nanmean(np.nanmean(power_epochs[subj_id][event][cond].copy().pick_channels(picks).data, axis=0), axis=0)

            im = ax[ix].imshow(plot_data,
                      extent=[times[0], times[-1], freqs[0], freqs[-1]], interpolation='Bicubic',
                      aspect='auto', origin='lower', cmap='RdBu_r', vmin = -np.nanmax(np.abs(plot_data)), vmax = np.nanmax(np.abs(plot_data)))
            ax[ix].set(yticks=yticks, xlabel='Time (s)', ylabel='Frequency', title=f'{subj_id}_{region}_{cond}_{event}')
            fig.colorbar(im, ax=ax[ix])
        #plt.savefig('/Users/alexandrafink/Documents/GraduateSchool/Classes/ML/final_proj/data/figs/decision5sec/'+subj_id+'_'+region+'_'+cond_name+'.png')


In [ ]:
## compute within trial mean for regression data
# data is n_epochs, n_elecs, n_freq, time


all_subj_epochs = []

event = 'DecisionOnset'
cond = 'regret'
freq_bands = np.logspace(*np.log10([4, 120]), num=20)


for ix, subj_id in enumerate(subj_ids):
    data = power_epochs[subj_id]['DecisionOnset'].to_data_frame()
    subj_num = ix 
    ch_names = power_epochs[subj_id]['DecisionOnset'].ch_names
    elec_names = [x.split('-')[0] for x in ch_names]
    for ch in ch_names:
        elec_name = ch.split('-')[0]
        for f in freq_bands:
            #freq_band = data.freqs[freq] #value of frequency
            for epoch in range(150):
                #idx = (subj_num+1)*epoch*(freq+1)
                y = power_epochs[subj_id]['DecisionOnset'].metadata.regret[epoch]
                all_subj_epochs.append({
                    'subj':subj_num,
                    'epoch':epoch,
                    'freq':f,
                    'elec':elec_name,
                    'av_pow': np.nanmean(data[ch][(data['freq'] == f) & (data['epoch']==epoch)],axis=0),
                    'regret':y_lab
                })
                

In [ ]:
epochs_df = pd.DataFrame(all_subj_epochs)